# Working with XNAT using PyXNAT

### Download PyXNAT Module

In [1]:
!pip install pyxnat

    100% |████████████████████████████████| 542kB 1.6MB/s ta 0:00:011
  Running setup.py bdist_wheel for pyxnat ... done
  Stored in directory: /Users/raminder/Library/Caches/pip/wheels/2d/79/f1/4a31fb18b992650bbd099574ce9675286db813ad3556282ccc
Successfully built pyxnat


#### Connect to server: Enter user : password 

In [86]:
import pyxnat
import os

# connect to XNAT instance
from pyxnat import Interface
xnat = Interface(server='http://sharp.bidmc.harvard.edu:8080',  cachedir='/tmp')
xnat.select.projects().get()

User: admin
········


['SHARP_1A', 'FAST', 'INSIGHT', 'SHARP_2B']

### Different types of datatypes supported by XNAT

In [3]:
xnat.inspect.datatypes()

['xnat:qcAssessmentData',
 'xnat:investigatorData',
 'xnat:qcManualAssessorData',
 'xnat:petmrSessionData',
 'xnat:subjectData',
 'xnat:crSessionData',
 'val:protocolData',
 'xnat:subjectVariablesData',
 'xnat:petSessionData',
 'xnat:pVisitData',
 'xnat:mrSessionData',
 'xnat:ctSessionData',
 'xnat:projectData',
 'xnat:eegScanData',
 'behavioral:scores',
 'xnat:eegSessionData']

### Check number of Subject

In [74]:
subjects = xnat.select('/projects/FAST/subjects')
subjects.get().__len__()

92

### Loading the project

In [19]:
project = xnat.select.project('FAST')
print(project)

<Project Object> FAST


### Working with Subject Data

In [80]:
contraints = [('xnat:subjectData/SUBJECT_ID','LIKE','%'),
                  'AND', ('xnat:subjectData/PROJECT', '=', 'FAST') 
              ]
table = xnat.select('xnat:subjectData', ['xnat:subjectData/PROJECT','xnat:subjectData/SUBJECT_ID','xnat:subjectData/SUBJECT_LABEL']).where(contraints)
table.__len__()

92

In [ ]:
print(table)

### Working with MRI Session Data

In [75]:
contraints = [('xnat:mrSessionData/ID','LIKE','%'),
                  'AND', ('xnat:subjectData/PROJECT', '=', 'FAST')
             ]
table1 = xnat.select('xnat:mrSessionData', ['xnat:mrSessionData/SUBJECT_LABEL','xnat:mrSessionData/SESSION_ID']).where(contraints)
table1.__len__()

159

In [82]:
print(table1)

subject_label,session_id
0001,SHARP_E00746
0001,SHARP_E00747
0002,SHARP_E00467
0002,SHARP_E00463
0003,SHARP_E00664
0003,SHARP_E00663
0004,SHARP_E00008
0004,SHARP_E00001
0005,SHARP_E00752
0005,SHARP_E00765
0007,SHARP_E00140
0007,SHARP_E00141
0008,SHARP_E00639
0008,SHARP_E00640
0009,SHARP_E00147
0009,SHARP_E01907
0010,SHARP_E00045
0010,SHARP_E00044
0011,SHARP_E00714
0011,SHARP_E00713
0013,SHARP_E00766
0013,SHARP_E00767
0014,SHARP_E00475
0015,SHARP_E00741
0015,SHARP_E00742
0016,SHARP_E00009
0016,SHARP_E00010
0017,SHARP_E00643
0017,SHARP_E00644
0019,SHARP_E00245
0020,SHARP_E00471
0021,SHARP_E00033
0022,SHARP_E00651
0022,SHARP_E01648
0023,SHARP_E00526
0023,SHARP_E00522
0024,SHARP_E00684
0024,SHARP_E00685
0025,SHARP_E00721
0026,SHARP_E00628
0026,SHARP_E00632
0030,SHARP_E00351
0030,SHARP_E00355
0031,SHARP_E00740
0031,SHARP_E00739
0032,SHARP_E00496
0032,SHARP_E00500
0034,SHARP_E00058
0034,SHARP_E00057
0036,SHARP_E00407
0037,SHARP_E00360
0037,SHARP_E00363
0038,SHARP_E00668
0038,SHARP_E00667
004

### Filtering using Behavioral scores

In [84]:
contraints = [('xnat:subjectData/SUBJECT_ID','LIKE','%'),
                  ('behavioral:scores/VocabScore', '>=', '36'),
                  'AND', ('xnat:subjectData/PROJECT', '=', 'FAST')
             ]
table1 = xnat.select('xnat:subjectData').where(contraints)
table1.__len__()

18

### Downloading the selective data

Lets start with download data for one subject

In [88]:
subject = xnat.select.project('FAST').subject('0001')
experiment = subject.experiment("SHARP_E00746")
allscans = experiment.scans()
allscans.download("/tmp", type='ALL', extract=False)

'/tmp/FAST_0001_SHARP_E00746_scans_ALL.zip'

Now lets write a filer to download the selective data for all the subjects

In [ ]:
# Filer can be developed based on the data parameters
contraints = [('xnat:mrSessionData/ID','LIKE','%'),
                  'AND', ('xnat:subjectData/PROJECT', '=', 'FAST')

list_subjects = xnat.select.project('FAST').subjects().where(contraints)
for list_subject in list_subjects:
    list_experiments = list_subject.experiments().where(contraints)
    for list_experiment in list_experiments:
        print list_experiment
        scans = list_experiment.scans()
        try:
            # Number 2 is for Anatomical data. Similar types can be set for other data types
            scans.download("/tmp", type='2', extract=False)
        except:
            print "There are no scans to download"